# 네이버 환율 정보 스크래이핑
나라, 등락, 변동, 환율 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup as BS

In [2]:
NAVER_FINANCE_URL = "https://finance.naver.com/marketindex/index.naver"

In [3]:
response = requests.get(NAVER_FINANCE_URL)
html_code = response.content
soup = BS(html_code, "html.parser")
soup


<script language="javascript" src="/template/head_js.naver?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/js/jindo.1.5.3.element-text-patch.js" type="text/javascript"></script>
<div id="container" style="padding-bottom:0px;">
<div class="market_include">
<div class="market_data">
<div class="market1">
<div class="title">
<h2 class="h_market1"><span>환전 고시 환율</span></h2>
</div>
<!-- data -->
<div class="data">
<ul class="data_lst" id="exchangeList">
<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_dn">
<span class="value">1,298.90</span>
<span class="txt_krw

In [4]:
crawl = soup.select_one(".data_lst")
crawl

<ul class="data_lst" id="exchangeList">
<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_dn">
<span class="value">1,298.90</span>
<span class="txt_krw"><span class="blind">원</span></span>
<span class="change"> 3.10</span>
<span class="blind">하락</span>
</div>
</a>
<a class="graph_img" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdc', '', '', event);">
<img alt="" height="153" src="https://ssl.pstatic.net/imgfinance/chart/marketindex/FX_USDKRW.png" width="295"/>
</a>
<div class="graph_info">
<span class="time">2023.03.28 13:52</span>
<span class="source">하나은행 기준</span>
<span class="count">고시회차<span class="num">472</span>회</span>
</div>
</li>
<li class="">
<a class="head jpy" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_JPYKRW" onclick="clic

In [22]:
국가 = crawl.select(".h_lst > .blind")
등락 = crawl.select("div > span.blind")
등락폭 = crawl.select(".change")
환율 = crawl.select(".value")
json_array = []

In [23]:
국가

[<span class="blind">미국 USD</span>,
 <span class="blind">일본 JPY(100엔)</span>,
 <span class="blind">유럽연합 EUR</span>,
 <span class="blind">중국 CNY</span>]

In [24]:
등락

[<span class="blind">하락</span>,
 <span class="blind">상승</span>,
 <span class="blind">상승</span>,
 <span class="blind">하락</span>]

In [25]:
등락폭

[<span class="change"> 3.10</span>,
 <span class="change">3.17</span>,
 <span class="change">1.20</span>,
 <span class="change"> 0.30</span>]

In [26]:
환율

[<span class="value">1,298.90</span>,
 <span class="value">994.45</span>,
 <span class="value">1,404.50</span>,
 <span class="value">188.69</span>]

In [27]:
for i in range(len(국가)):
  json_array.append({})
  json_array[i]["국가"] = 국가[i].text
  json_array[i]["등락"] = 등락[i].text
  json_array[i]["등락폭"] = 등락폭[i].text
  json_array[i]["환율"] = 환율[i].text
json_array

[{'국가': '미국 USD', '등락': '하락', '등락폭': ' 3.10', '환율': '1,298.90'},
 {'국가': '일본 JPY(100엔)', '등락': '상승', '등락폭': '3.17', '환율': '994.45'},
 {'국가': '유럽연합 EUR', '등락': '상승', '등락폭': '1.20', '환율': '1,404.50'},
 {'국가': '중국 CNY', '등락': '하락', '등락폭': ' 0.30', '환율': '188.69'}]

In [30]:
import pandas as pd
table = pd.DataFrame(json_array)

In [39]:
table.to_csv("data.csv", encoding="cp949")

---
# 새로운 방법 찾기

In [55]:
lst = crawl.select(".head")
dict_lst = []
for i in range(len(lst)):
  dict_lst.append({})
  dict_lst[i]["국가"] = lst[i].select_one(".h_lst > .blind").text
  dict_lst[i]["등락"] = lst[i].select_one(".head_info > .blind").text
  dict_lst[i]["등락폭"] = float(lst[i].select_one(".head_info > .change").text)
  dict_lst[i]["환율"] = float(lst[i].select_one(".head_info > .value").text.replace(",",""))
data = pd.DataFrame(dict_lst)
data.to_csv("float_data.csv", encoding="cp949")